In [27]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import scipy.integrate as integrate


In [2]:
data_df = pd.read_csv('data/Project-Data.csv')
data_df.head()

In [16]:
sigma_r = 0.00023

maturities = data_df['Maturity'].values
zc_yields = data_df['ZCYield'].values
futures_prices = data_df['FuturesPrice'].values

## Bond pricing

In [5]:
def b0(T, t, lambda_r, sigma_r, r_bar):
    return (np.exp(-2 * lambda_r * (T-t)) * (np.exp(lambda_r * (T-t)) - 1) *
            (sigma_r**2 + np.exp(lambda_r * (T-t)) * (4 * lambda_r**2 * r_bar - 3 * sigma_r**2)) +
            2 * lambda_r * (T-t) * (sigma_r**2 - 2 * lambda_r**2 * r_bar)) / (4 * lambda_r**3)

def br(T, t, lambda_r):
    return (np.exp(-lambda_r * (T-t)) - 1) / lambda_r

In [6]:
initial_guess = [0.1, 0.03, 0.01]

### Least Squares Calibration

In [7]:
def least_squares_objective(params):
    lambda_r, r_bar, r0 = params
    error = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        error += (P_model - P_market) ** 2
    return error

least_squares_result = minimize(least_squares_objective, initial_guess, method='Nelder-Mead')
lambda_r_ls, r_bar_ls, r0_ls = least_squares_result.x
print(f'Least Squares Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_ls}')
print(f'Long-term mean (r_bar): {r_bar_ls}')
print(f'Initial value (r0): {r0_ls}')

Least Squares Calibration:
Mean reversion intensity (lambda_r): 0.2991437388948194
Long-term mean (r_bar): 0.010033839633324947
Initial value (r0): 0.014603967286584005


### Maximum Likelihood Calibration

In [8]:
def log_likelihood(params):
    lambda_r, r_bar, r0 = params
    likelihood = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        likelihood += norm.logpdf(P_market, loc=P_model, scale=sigma_r)
    return -likelihood  # Negative log-likelihood

mle_result = minimize(log_likelihood, initial_guess, method='Nelder-Mead')
lambda_r_mle, r_bar_mle, r0_mle = mle_result.x
print(f'Maximum Likelihood Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_mle}')
print(f'Long-term mean (r_bar): {r_bar_mle}')
print(f'Initial value (r0): {r0_mle}')

Maximum Likelihood Calibration:
Mean reversion intensity (lambda_r): 0.2991437388948194
Long-term mean (r_bar): 0.010033839633324947
Initial value (r0): 0.014603967286584005


### Long Term Quantile Method

In [9]:
def quantile_objective(params):
    lambda_r, r_bar, r0 = params
    quantile_error = 0
    for maturity, yield_ in zip(maturities, zc_yields):
        T = maturity
        t = 0  # Assuming current time is 0
        P_model = np.exp(b0(T, t, lambda_r, sigma_r, r_bar) + br(T, t, lambda_r) * r0)
        P_market = np.exp(-yield_ * (T-t))
        quantile_error += np.abs(P_model - P_market)  # Absolute error for simplicity
    return quantile_error

# Perform the optimization
quantile_result = minimize(quantile_objective, initial_guess, method='Nelder-Mead')

# Extract the calibrated parameters
lambda_r_quantile, r_bar_quantile, r0_quantile = quantile_result.x

print(f'Long Term Quantile Calibration:')
print(f'Mean reversion intensity (lambda_r): {lambda_r_quantile}')
print(f'Long-term mean (r_bar): {r_bar_quantile}')
print(f'Initial value (r0): {r0_quantile}')

Long Term Quantile Calibration:
Mean reversion intensity (lambda_r): 0.16322924742036396
Long-term mean (r_bar): 0.008550775735612639
Initial value (r0): 0.01429913713437552


## Futures pricing

In [20]:
init_spot_price_oil_pb = 80

def calculate_volatility_and_correlation(matrix):
    var_r = matrix[0, 0]
    var_delta = matrix[1, 1]
    var_log_O = matrix[2, 2]
    
    cov_r_delta = matrix[0, 1]
    cov_r_log_O = matrix[0, 2]
    cov_delta_log_O = matrix[1, 2]
    
    sigma_r = np.sqrt(var_r)
    sigma_delta = np.sqrt(var_delta)
    sigma_O = np.sqrt(var_log_O)
    
    rho_r_delta = cov_r_delta / (sigma_r * sigma_delta)
    rho_r_log_O = cov_r_log_O / (sigma_r * sigma_O)
    rho_delta_log_O = cov_delta_log_O / (sigma_delta * sigma_O)
    
    return sigma_r, sigma_delta, sigma_O, rho_r_delta, rho_r_log_O, rho_delta_log_O
    
# Quadratic variation matrix
matrix = np.array([
    [0.000529, 0.00003312, 0.44712],
    [0.00003312, 0.000144, 0.08352],
    [0.44712, 0.08352, 576]
]) * 1e-4

# Calculate volatility and correlation
sigma_r, sigma_delta, sigma_O, rho_r_delta, rho_r_log_O, rho_delta_log_O = calculate_volatility_and_correlation(matrix)
print(f'Sigma_r: {sigma_r}')
print(f'Sigma_delta: {sigma_delta}')
print(f'Sigma_O: {sigma_O}')

Sigma_r: 0.00023
Sigma_delta: 0.00012
Sigma_O: 0.24000000000000002


In [68]:
def futures_price(t, T, rt, delta_t, Ot, phi_0, phi_r, phi_delta):
    exponent = phi_0 * (T - t) + \
        phi_r * (T - t) * rt + \
        phi_delta * (T - t) * delta_t        
    return np.exp(exponent) * Ot

def phi_r(tau, lambda_r):
    return (1 - np.exp(-lambda_r * tau)) / lambda_r

def phi_delta(tau, lambda_delta):
    return (np.exp(-lambda_delta * tau) - 1) / lambda_delta

def phi_0(tau, lambda_r, lambda_delta, sigma_r, sigma_delta, sigma_O, bar_delta, bar_r):
    def integrand(s):
        phi_delta_s = phi_delta(s, lambda_delta)
        phi_r_s = phi_r(s, lambda_r)
        term1 = phi_delta_s * (2 * lambda_delta * bar_delta + (np.linalg.norm(sigma_O) ** 2) * phi_delta_s + 2 * np.dot(sigma_delta.T, sigma_O))
        term2 = 2 * phi_r_s * (lambda_r * bar_r + phi_delta_s * np.dot(sigma_r.T, sigma_O) + np.dot(sigma_r.T, sigma_delta))
        term3 = (np.linalg.norm(sigma_r) ** 2) * (phi_r_s ** 2)
        return 0.5 * (term1 + term2 + term3)
    
    phi0, _ = integrate.quad(integrand, 0, tau)
    return phi0

In [76]:
def objective(params, futures_prices, maturities, r0, r_bar, lambda_r, sigma_r, sigma_delta,sigma_O, O0):
    lambda_delta, bar_delta, delta_0 = params
    r_t = r0 
    errors = []
    for T, f_observed in zip(maturities, futures_prices):
        t = 0
        phi_delta_ = phi_delta(T, lambda_delta)
        phi_r_ = phi_r(T, lambda_r)
        phi_0_ = phi_0(T, lambda_r, lambda_delta, sigma_r, sigma_delta, sigma_O, bar_delta, r_bar)
        f_theoretical = futures_price(t, T, r_t, delta_0, O0 , phi_0_, phi_r_, phi_delta_)
        # print(f'Theoretical Futures Price: {f_theoretical}')
        # print(f'Observed Futures Price: {f_observed}\n')
        error = f_observed - f_theoretical
        errors.append(error)
    
    # return np.sum(np.abs(errors))
    return np.sum(np.square(errors))

# Initial guesses for parameters
initial_guess = [0.3, 0.02, 0.01]

# Minimize the objective function
args=(futures_prices, maturities, r0_mle, r_bar_mle, lambda_r_mle, sigma_r, sigma_delta, sigma_O, init_spot_price_oil_pb)
result = minimize(objective, initial_guess, args=args , method='Nelder-Mead')

# Extract the calibrated parameters
lambda_delta, bar_delta, delta_0 = result.x

print(f"Calibrated parameters:")
print(f"lambda_delta: {lambda_delta}")
print(f"bar_delta: {bar_delta}")
print(f"delta_0: {delta_0}")


Calibrated parameters:
lambda_delta: 1.0368961713229132
bar_delta: 0.03625173630807739
delta_0: -0.0013515837557829763


## Storage options